In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import f_oneway, shapiro, levene, kruskal, sem
import scipy.stats as st 
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import AnovaRM
import scikit_posthocs as sp
import numpy as np
import string
from matplotlib.backends.backend_pdf import PdfPages


In [3]:
# Manditory column order ['date', 'country', 'Subject', 'events']
dfc = pd.read_csv('search_data_pest.csv')
sub = dfc.columns[2]
print(dfc.columns)

Index(['date', 'country', 'pest', 'events'], dtype='object')


In [4]:
# Format date column
dfc['date'] = pd.to_datetime(dfc['date'], format='%d-%b-%y', dayfirst=True)
print(dfc.head())


        date country                           pest  events
0 2023-09-30   India                     Leaf miner       1
1 2023-09-30    Peru              Planococcus citri       1
2 2023-09-30   India                Citrus mealybug       1
3 2023-09-30   India  Brinjal fruit and shoot borer       1
4 2023-09-30  Canada                         Thrips       1


<font size="5"> Compact letter display algorithm</font>

In [5]:
def cld_al(df, alpha=0.1):

    df["p-adj"] = df["p-adj"].astype(float)

    # Creating a list of the different treatment groups from Tukey's
    group1 = set(df.group1.tolist())  # Dropping duplicates by creating a set
    group2 = set(df.group2.tolist())  # Dropping duplicates by creating a set
    groupSet = group1 | group2  # Set operation that creates a union of 2 sets
    groups = sorted(list(groupSet))

    # Creating lists of letters that will be assigned to treatment groups
    letters = list(string.ascii_lowercase)[:len(groups)]
    cldgroups = letters

    # the following algoritm is a simplification of the classical cld,

    cld = pd.DataFrame(list(zip(groups, letters, cldgroups)))
    cld[3]=""
    
    for row in df.itertuples():
        if df["p-adj"][row[0]] > (alpha):
            cld.iat[groups.index(df["group1"][row[0]]), 2] += cld.iat[groups.index(df["group2"][row[0]]), 1]
            cld.iat[groups.index(df["group2"][row[0]]), 2] += cld.iat[groups.index(df["group1"][row[0]]), 1]
            
        if df["p-adj"][row[0]] < (alpha):
                cld.iat[groups.index(df["group1"][row[0]]), 3] +=  cld.iat[groups.index(df["group2"][row[0]]), 1]
                cld.iat[groups.index(df["group2"][row[0]]), 3] +=  cld.iat[groups.index(df["group1"][row[0]]), 1]

    cld[2] = cld[2].apply(lambda x: "".join(sorted(x)))
    cld[3] = cld[3].apply(lambda x: "".join(sorted(x)))
    cld.rename(columns={0: "groups"}, inplace=True)

    # this part will reassign the final name to the group
    # for sure there are more elegant ways of doing this
    cld = cld.sort_values(cld.columns[2], key=lambda x: x.str.len())
    cld["labels"] = ""
    letters = list(string.ascii_lowercase)
    unique = []
    for item in cld[2]:

        for fitem in cld["labels"].unique():
            for c in range(0, len(fitem)):
                if not set(unique).issuperset(set(fitem[c])):
                    unique.append(fitem[c])
        g = len(unique)

        for kitem in cld[1]:
            if kitem in item:
                if cld["labels"].loc[cld[1] == kitem].iloc[0] == "":
                    cld["labels"].loc[cld[1] == kitem] += letters[g]

                #Checking if there are forbidden pairing (proposition of solution to the imperfect script)                
                if kitem in ' '.join(cld[3][cld["labels"]==letters[g]]): 
                    g=len(unique)+1
               
                # Checking if columns 1 & 2 of cld share at least 1 letter
                if len(set(cld["labels"].loc[cld[1] == kitem].iloc[0]).intersection(cld.loc[cld[2] == item, "labels"].iloc[0])) <= 0:
                    if letters[g] not in list(cld["labels"].loc[cld[1] == kitem].iloc[0]):
                        cld["labels"].loc[cld[1] == kitem] += letters[g]
                    if letters[g] not in list(cld["labels"].loc[cld[2] == item].iloc[0]):
                        cld["labels"].loc[cld[2] == item] += letters[g]

    cld = cld.sort_values("labels")
    #print(cld)
    #print('\n')
    cld.drop(columns=[1, 2, 3], inplace=True)
    #print(cld)
    #print('\n')
    #print('\n')
    return(cld)


<font size="5"> ANOVA loop</font>

In [26]:
# Create df ANOVA for results  
test_results = pd.DataFrame(columns=['test', 'country', sub, 'F/H-statistic', 'p-value'])


country_index = 0


# Testing loop 
for country in dfc['country'].unique():
    filtered_country_df = dfc[dfc['country'] == country]
    country_index += 1
    print('country ', country_index, '/', len(dfc['country'].unique()), ' Started.')
    sub_index = 0
    
    for subject in filtered_country_df[sub].unique():
        filtered_df = filtered_country_df[filtered_country_df[sub] == subject]
        sub_index +=1
        print('Subject ', sub_index,'/', len(filtered_country_df[sub].unique()), 'in country ', country_index,'/',len(dfc['country'].unique()), ' Started.')

        # Test data assumptions/requirements
        if len(filtered_df['date'].dt.month.unique()) >= 3:
            print('groups for ', country, 'and ', subject, 'are sufficient')
            # Specify the date range
            start_date = '2021-10-01'
            end_date = '2023-09-30'
            date_range = pd.date_range(start=start_date, end=end_date, freq='D')

            # Create a df with the date range
            date_range_df = pd.DataFrame({'date': date_range})
 
            # Merge existing data with the date range, filling missing values with zeros and specified crop name
            filtered_df = pd.merge(date_range_df, filtered_df, on='date', how='left').fillna({'events': 0, sub : subject, 'country': country})
            filtered_df['month'] = filtered_df['date'].dt.month
            filtered_df['year'] = filtered_df['date'].dt.year

            
            #Average data by month 
            filtered_df['events'] = pd.to_numeric(filtered_df['events'], errors='coerce')
            
            # Check normality
            _, p_normality = shapiro(filtered_df['events'])
            print(country , subject , p_normality)
            # Check homogeneity of variances
            _, p_homogeneity = levene(filtered_df['events'], filtered_df['month'])
            print(country , subject , p_homogeneity)

            # check p values and proceed as required
            if p_normality < 0.05 and p_homogeneity < 0.05:

                
                # The data assumptions are met so proceed with ANOVA test 
                print('ANOVA performed for ', country, 'and ', subject,)
                model = AnovaRM(filtered_df, 'events', 'year', within=['month'], aggregate_func='sum')
                results = model.fit()
                print(results.summary())
                first_row = results.anova_table.iloc[0]
                f_statistic = first_row['F Value']
                print(f_statistic)
                p_value = p_value = first_row['Pr > F']
                print(p_value)
                dict_res ={ 
                    'test': 'ANOVA',
                    'country': country,
                    sub : subject,
                    'F/H-statistic': float(f_statistic),
                    'p-value': float(p_value)
                }
                test_results = (
                    pd.DataFrame([dict_res]).copy() if test_results.empty 
                    else pd.concat([test_results, pd.DataFrame([dict_res])], ignore_index=True)
                )
            
            #The data assumptions are not met so proceed with the Kruskal-Wallis test 
            else:
                #Proceed with the Kruskal-Wallis test 
                print('kruskal performed for ', country, 'and ', subject,)
                h_statistic, p_value_kruskal = kruskal(filtered_df['month'], filtered_df['events'])
                #print(p_value_kruskal)
                dict_res ={ 
                    'test': 'Kruskal',
                    'country': country,
                    sub : subject,
                    'F/H-statistic': float(h_statistic),
                    'p-value': float(p_value_kruskal)
                }
             
                test_results = (
                    pd.DataFrame([dict_res]).copy() if test_results.empty 
                    else pd.concat([test_results, pd.DataFrame([dict_res])], ignore_index=True)
                )
                
        # Not enough samples to run the test
        else:
            print('not enough for ', country, 'and ', subject)
            dict_res ={ 
                    'test': 'Not enough data',
                    'country': country,
                    sub : subject,
                    'F/H-statistic': 0,
                    'p-value': 0
                }
            
            test_results = (
                    pd.DataFrame([dict_res]).copy() if test_results.empty 
                    else pd.concat([test_results, pd.DataFrame([dict_res])], ignore_index=True)
                )

test_results.to_csv(sub+' results.csv', index=False)
print('complete')

country  1 / 165  Started.
Subject  1 / 473 in country  1 / 165  Started.
groups for  India and  Leaf miner are sufficient
India Leaf miner 1.401298464324817e-45
India Leaf miner 1.3809306101583667e-250
ANOVA performed for  India and  Leaf miner
               Anova
      F Value  Num DF  Den DF Pr > F
------------------------------------
month  2.7969 11.0000 10.0000 0.0582

2.7969303423848872
0.058152810997791744
Subject  2 / 473 in country  1 / 165  Started.
groups for  India and  Citrus mealybug are sufficient
India Citrus mealybug 0.0
India Citrus mealybug 7.726776488875463e-269
ANOVA performed for  India and  Citrus mealybug
               Anova
      F Value  Num DF  Den DF Pr > F
------------------------------------
month  0.7241 11.0000 10.0000 0.6986

0.724054706355591
0.6986084142436138
Subject  3 / 473 in country  1 / 165  Started.
groups for  India and  Brinjal fruit and shoot borer are sufficient
India Brinjal fruit and shoot borer 1.401298464324817e-44
India Brinjal frui

<font size="5"> Tukey loop (ANOVA follow up)</font>

In [30]:
#ANOVA follow up
anova_results = test_results[test_results['test'] == 'ANOVA']
tukey_results = pd.DataFrame(columns=['country', sub, 'group1', 'group2', 'meandiff', 'p-adj', 'lower', 'upper', 'reject'])
graph_data_df = pd.DataFrame(columns=['country', sub,'month', 'events_mean', 'events_std','confidence_interval', 'labels', 'groups'])
test_index = 0

for index, row in anova_results.iterrows():
    
    test_index += 1 
    print('test ', test_index, '/', len(anova_results), ' started')
    
    p_value = row['p-value']
    
    if p_value < 0.1:
        
        filtered_anova_df = dfc[(dfc['country'] == row['country']) & (dfc[sub] == row[sub])]
        temp_country = row['country']
        temp_crop = row[sub]
      
        # Specify the date range
        start_date = '2021-10-01'
        end_date = '2023-09-30'
        date_range = pd.date_range(start=start_date, end=end_date, freq='D')

        # Create a df with the date range
        date_range_df = pd.DataFrame({'date': date_range})
 
        # Merge existing data with the date range, filling missing values with zeros and specified crop name
        filtered_anova_df = pd.merge(date_range_df, filtered_anova_df, on='date', how='left').fillna({'events': 0, sub : row[sub], 'country': row['country']})
        filtered_anova_df['month'] = filtered_anova_df['date'].dt.month
        filtered_anova_df['year'] = filtered_anova_df['date'].dt.year

        # Agregate data by month 
        filtered_anova_df['events'] = pd.to_numeric(filtered_anova_df['events'], errors='coerce')
        filtered_anova_df = filtered_anova_df.groupby(['year', 'month'])['events'].sum().reset_index()
        filtered_anova_df.sort_values(by='month', inplace=True)

        #prep graph data 
        graph_data = filtered_anova_df.groupby('month')['events'].agg(['mean', 'std', 'count', 'sem']).reset_index()
        graph_data['samp_count'] = graph_data['month'].map(filtered_anova_df['month'].value_counts())
        graph_data.columns = ['month', 'events_mean', 'events_std', 'count', 'sem', 'samp_count']
        graph_data['confidence_interval'] = graph_data.apply(
    lambda row: st.t.interval(0.90, loc=row['events_mean'], scale=row['sem'], df=row['samp_count']-1) if row['events_mean'] != 0 else (0, 0),
    axis=1
)
        print(temp_country + " " + temp_crop)
        print(graph_data.head(12))
              
        #scale=row['events_std'] / np.sqrt(row['count'])), axis=1)
        graph_data['country'] = temp_country
        graph_data[sub] = temp_crop
        

        # run Tukey test
        tukey = pairwise_tukeyhsd(endog=filtered_anova_df['events'], groups=filtered_anova_df['month'], alpha=0.1)
        if tukey.reject.any():
            print(row['country'], row[sub], ' valid')
            tukey_df = pd.DataFrame(data=tukey._results_table.data[1:], columns=tukey._results_table.data[0])
            tukey_df['country'] = row['country']
            tukey_df[sub] = row[sub]
            

            # Get the result summary as a DataFrame
            tukey_summary = pd.DataFrame(data=tukey.summary().data[1:], columns=tukey.summary().data[0])

            #CLD algarythem 
            cld_data = cld_al(tukey_summary, alpha=0.1)

            #merge CLD with data 
            merged_df = pd.merge(graph_data, cld_data, left_on='month', right_on='groups')
            merged_df = merged_df[['country', sub ,'month', 'events_mean', 'events_std','confidence_interval', 'labels', 'groups']]
            

            #concatonate tables
            tukey_results = (tukey_df.copy() if tukey_results.empty else pd.concat([tukey_results, tukey_df], ignore_index=True))
            graph_data_df = (merged_df.copy() if graph_data_df.empty else pd.concat([graph_data_df, merged_df], ignore_index=True))

            
            
        else:
            
            print(row['country'], row[sub] + ' False')


tukey_results.to_csv(sub+' tukey_results.csv', index=False)

print('complete')
        

test  1 / 1390  started
India Leaf miner
    month  events_mean  events_std  count  sem  samp_count  \
0       1          5.0    7.071068      2  5.0           2   
1       2          6.5    9.192388      2  6.5           2   
2       3          8.5   12.020815      2  8.5           2   
3       4          3.5    4.949747      2  3.5           2   
4       5          3.5    4.949747      2  3.5           2   
5       6          1.0    1.414214      2  1.0           2   
6       7          2.0    1.414214      2  1.0           2   
7       8          2.0    2.828427      2  2.0           2   
8       9          4.5    0.707107      2  0.5           2   
9      10          6.0    8.485281      2  6.0           2   
10     11          3.5    4.949747      2  3.5           2   
11     12          3.5    4.949747      2  3.5           2   

                          confidence_interval  
0     (-26.5687575740047, 36.568757574004664)  
1    (-34.539384846206104, 47.53938484620606)  
2     (-

C:\Users\DownsF\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\DownsF\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


India Thrips False
test  11 / 1390  started
test  12 / 1390  started
test  13 / 1390  started
test  14 / 1390  started
India Grasshopper
    month  events_mean  events_std  count  sem  samp_count  \
0       1          0.5    0.707107      2  0.5           2   
1       2          1.0    1.414214      2  1.0           2   
2       3          0.0    0.000000      2  0.0           2   
3       4          2.0    0.000000      2  0.0           2   
4       5          0.0    0.000000      2  0.0           2   
5       6          1.0    1.414214      2  1.0           2   
6       7          0.0    0.000000      2  0.0           2   
7       8          0.0    0.000000      2  0.0           2   
8       9          1.0    1.414214      2  1.0           2   
9      10          0.0    0.000000      2  0.0           2   
10     11          0.5    0.707107      2  0.5           2   
11     12          1.5    2.121320      2  1.5           2   

                         confidence_interval  
0   (-2.6

C:\Users\DownsF\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\DownsF\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


India Grasshopper False
test  15 / 1390  started
test  16 / 1390  started
India Achaea janata
    month  events_mean  events_std  count   sem  samp_count  \
0       1          3.0    4.242641      2   3.0           2   
1       2          5.0    7.071068      2   5.0           2   
2       3          4.5    6.363961      2   4.5           2   
3       4         10.5   14.849242      2  10.5           2   
4       5          6.0    8.485281      2   6.0           2   
5       6          7.0    9.899495      2   7.0           2   
6       7          6.0    7.071068      2   5.0           2   
7       8          4.5    6.363961      2   4.5           2   
8       9          6.0    2.828427      2   2.0           2   
9      10          3.0    4.242641      2   3.0           2   
10     11          8.5   12.020815      2   8.5           2   
11     12          5.0    7.071068      2   5.0           2   

                          confidence_interval  
0   (-15.941254544402817, 21.941254544

...

C:\Users\DownsF\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\DownsF\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


Netherlands Thrips False
test  1239 / 1390  started
test  1240 / 1390  started
test  1241 / 1390  started
test  1242 / 1390  started
test  1243 / 1390  started
test  1244 / 1390  started
test  1245 / 1390  started
test  1246 / 1390  started
test  1247 / 1390  started
test  1248 / 1390  started
test  1249 / 1390  started
test  1250 / 1390  started
test  1251 / 1390  started
test  1252 / 1390  started
test  1253 / 1390  started
test  1254 / 1390  started
test  1255 / 1390  started
test  1256 / 1390  started
test  1257 / 1390  started
test  1258 / 1390  started
test  1259 / 1390  started
test  1260 / 1390  started
test  1261 / 1390  started
test  1262 / 1390  started
test  1263 / 1390  started
test  1264 / 1390  started
test  1265 / 1390  started
Belgium Mealybug
    month  events_mean  events_std  count  sem  samp_count  \
0       1          0.0    0.000000      2  0.0           2   
1       2          0.0    0.000000      2  0.0           2   
2       3          0.0    0.000000      2  

C:\Users\DownsF\AppData\Local\Temp\ipykernel_31756\1052515824.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cld["labels"].loc[cld[1] == kitem] += letters[g]
C:\Users\DownsF\AppData\Local\Temp\ipykernel_31756\1052515824.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cld["labels"].loc[cld[1] == kitem] += letters[g]
C:\Users\DownsF\AppData\Local\Temp\ipykernel_31756\1052515824.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

test  1308 / 1390  started
test  1309 / 1390  started
test  1310 / 1390  started
test  1311 / 1390  started
test  1312 / 1390  started
test  1313 / 1390  started
test  1314 / 1390  started
test  1315 / 1390  started
test  1316 / 1390  started
test  1317 / 1390  started
test  1318 / 1390  started
test  1319 / 1390  started
Burkina Faso No Data
    month  events_mean  events_std  count  sem  samp_count  \
0       1          0.0    0.000000      2  0.0           2   
1       2          0.0    0.000000      2  0.0           2   
2       3          0.0    0.000000      2  0.0           2   
3       4          0.0    0.000000      2  0.0           2   
4       5          0.0    0.000000      2  0.0           2   
5       6          0.0    0.000000      2  0.0           2   
6       7          0.0    0.000000      2  0.0           2   
7       8          0.0    0.000000      2  0.0           2   
8       9          1.0    1.414214      2  1.0           2   
9      10          0.5    0.707107 

C:\Users\DownsF\AppData\Local\Temp\ipykernel_31756\1052515824.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cld["labels"].loc[cld[1] == kitem] += letters[g]
C:\Users\DownsF\AppData\Local\Temp\ipykernel_31756\1052515824.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cld["labels"].loc[cld[1] == kitem] += letters[g]
C:\Users\DownsF\AppData\Local\Temp\ipykernel_31756\1052515824.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

<font size="5"> Plot graphs with CLD and confidence intervals</font>

In [32]:
#create plots and save them to pdf
pdffile = PdfPages(sub+' graph.pdf')

#graph_data_df.sort_values(by=sub, inplace=True)
plt.figure()

country_index = 0
print(graph_data_df.head())

for country in graph_data_df['country'].unique():
    country_index += 1
    print('country ', country_index, '/', len(graph_data_df['country'].unique()), ' Started.')
    sub_index = 0

    temp_filterd_graph_df = graph_data_df[graph_data_df['country'] == country]
    

    for subject in temp_filterd_graph_df[sub].unique():     
        sub_index +=1
        print('Subject ', sub_index,'/', len(temp_filterd_graph_df[sub].unique()), 'in country ', country_index,'/',len(graph_data_df['country'].unique()), ' Started.')
        
        filterd_graph_df = temp_filterd_graph_df [temp_filterd_graph_df[sub] == subject]

        
        x_values = filterd_graph_df['month']
        y_values = filterd_graph_df['events_mean']
        std_values = filterd_graph_df['events_std']
        data_label = filterd_graph_df['labels']
        lower_ci = filterd_graph_df['confidence_interval'].apply(lambda x: x[0])
        upper_ci = filterd_graph_df['confidence_interval'].apply(lambda x: x[1])
        confidence_interval = [y_values - lower_ci, upper_ci - y_values]
        standard_error = filterd_graph_df['sem']

        # Plotting the bar chart with confidence intervals as error bars
        plt.bar(x_values, y_values, yerr = standard_error, capsize=5, label='Events', color='blue')
                
        # Adding data labels
        for x, y, label in zip(x_values, y_values, data_label):
            plt.text(x, y + 0.02, f'{label}', ha='center', va='bottom') 

        # Adding labels and title    
        plt.xlabel('Month')
        plt.ylabel('Events')
        plt.title(country + subject)
        plt.legend()
        pdffile.savefig()
        plt.close()
    
pdffile.close()
print('complete')

  country          pest  month  events_mean  events_std  \
0  Canada  Cabbage worm      1          0.0    0.000000   
1  Canada  Cabbage worm      2          5.5    2.121320   
2  Canada  Cabbage worm      3          0.0    0.000000   
3  Canada  Cabbage worm      4          2.0    0.000000   
4  Canada  Cabbage worm      5          2.0    2.828427   

                         confidence_interval labels  groups  sem  
0                                     (0, 0)      b       1  0.0  
1  (-3.9706272722014084, 14.970627272201398)      a       2  1.5  
2                                     (0, 0)      b       3  0.0  
3                                 (nan, nan)     ab       4  0.0  
4  (-10.627503029601879, 14.627503029601865)     ab       5  2.0  
country  1 / 14  Started.
Subject  1 / 9 in country  1 / 14  Started.
Subject  2 / 9 in country  1 / 14  Started.
Subject  3 / 9 in country  1 / 14  Started.
Subject  4 / 9 in country  1 / 14  Started.
Subject  5 / 9 in country  1 / 14  Starte